#### Set Up

In [1]:
import os
import json
import re
from tqdm import tqdm

from selfcheck import SelfCheckPrompt
from utils import load_data

c:\Users\ADMIN\.conda\envs\base-12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
intrinsic_prompt_path = 'prompts/selfcheck_intrinsic_vi.txt'
extrinsic_prompt_path = 'prompts/selfcheck_extrinsic_vi.txt'
general_prompt_path = 'prompts/selfcheck_original_vi.txt'
data_path = 'data/vihallu-train.csv'
output_path = 'data/output/selcheck_qwen4_4b_train.json'

In [6]:
selfcheck = SelfCheckPrompt(
    model_name='Qwen/Qwen3-4B-Instruct-2507',
    intrinsic_prompt_path=intrinsic_prompt_path,
    extrinsic_prompt_path=extrinsic_prompt_path,
    general_prompt_path=general_prompt_path,   
)

def extract_sentences(passage, min_length=10):
    sentence_pattern = r'[^.!?\n]+[.!?\n]?'
    raw_sentences = re.findall(sentence_pattern, passage)
    sentences = []
    for sentence in raw_sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        if sentences and len(sentence) < min_length:
            sentences[-1] = sentences[-1] + ' ' + sentence
        else:
            sentences.append(sentence)
    return sentences

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 82.23it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [4]:
data = load_data(data_path)
print(f"Length of train data: {len(data)}")
data.head()

Length of train data: 7000


,id,context,prompt,response,label
0,9b1ea51d-d1ff-45ba-8cf1-6a91328e8600,"Vào những năm 1870, hai nhà điêu khắc Augustus...","Vào những năm 1960, nơi nào trở thành trung tâ...",Quảng trường Washington là trung tâm của thế h...,extrinsic
1,db7a89c6-2a6a-42af-beef-58e557ecc819,Cách mạng Tháng Mười đã biến một cuộc chiến tr...,Cách mạng Tháng Mười đã khởi xướng chủ nghĩa p...,Sai. Cách mạng Tháng Mười đã đánh bại chủ nghĩ...,no
2,10fca062-d343-4eca-8434-93c7a8aa5e0e,Vị trí địa lý nằm giữa phương Đông và Địa Trun...,Dựa trên việc Đế quốc Ottoman không bao giờ ki...,Nền ẩm thực của Thổ Nhĩ Kỳ được xây dựng từ gi...,extrinsic
3,ece8eb9e-d6bb-407a-a567-d9531861c603,"Hồ Quý Ly trước có tên là Lê Quý Ly (黎季犛), biể...",Đời cháu thứ 12 của Hồ Hưng Dật sống ở đâu?,"Đời cháu thứ 12 của Hồ Hưng Dật, là Hồ Liêm, s...",no
4,b613217f-df2a-491e-8326-25811a31eb09,"Năm 2007, Bồ Đào Nha có khoảng 332.137 người n...","Từ thập niên 1990, tại sao xuất hiện một vài l...",Mặc dù Bồ Đào Nha đã đóng cửa biên giới hoàn t...,intrinsic


#### Benchmark

In [1]:
import spacy
nlp = spacy.load('vi_core_news_md')

text = "Sai. Tại Texas, chỉ có gần 38.000 người Cuba sinh sống. Thực tế, số lượng người Cuba tại Texas ít hơn nhiều so với người gốc México, những người chiếm phần lớn dân cư gốc Mỹ Latinh trong bang."

doc = nlp(text)
sentences = list(doc.sents)

for sentence in sentences:
    print(sentence.text)


OSError: [E050] Can't find model 'vi_core_news_md'. It doesn't seem to be a Python package or a valid path to a data directory.

In [5]:
data = data[:10]
result = {}

for id, sample in tqdm(data.iterrows(), total=len(data), desc="Processing samples"):
    sentences = extract_sentences(sample['response'])
    result[sample['id']] = {}
    result[sample['id']]['sentences'] = {}
    result[sample['id']]['label'] = sample['label']
    
    for pos, sent in enumerate(sentences):
        label_general = selfcheck.predict_hallucination(
            context=sample['context'],
            sentence=sent,
        )
        label_intrinsic = selfcheck.predict_intrinsic(
            context=sample['context'],
            sentence=sent,
        )
        label_extrinsic = selfcheck.predict_extrinsic(
            context=sample['context'],
            sentence=sent,
        )
        
        result[sample['id']]['sentences'][pos] = {
            'sentence': sent,
            'general_label': label_general,
            'intrinsic_label': label_intrinsic,
            'extrinsic_label': label_extrinsic,
        }
        result

Processing samples:  10%|█         | 1/10 [00:07<01:05,  7.24s/it]


IndexError: list index out of range

In [ ]:
with open(output_path, 'w') as fout:
    json.dump(result, fout, indent=4)